In [ ]:
## Remove games that are not played by one of the 30 major league teams
## Figure out what's going on in your data...

In [1]:
import pandas as pd
import numpy as np
import requests

import json
from pandas.io.json import json_normalize
from datetime import timedelta, date
import time
import pickle
import glob

pd.set_option('display.max_columns', 500)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.neighbors import KNeighborsClassifier

def read_pickle(file):
    with open(file, 'rb') as picklefile:
        data = pickle.load(picklefile)
    return(data)

def write_pickle(file, data):
    with open(file, 'wb') as picklefile:
        pickle.dump(data, picklefile)
        

In [2]:
player_file_list = glob.glob('/Users/JacKuo14/Documents/Metis/course_work/project03_baseball/sportstradar_data2/*player*.pkl')
gamefile_list = glob.glob('/Users/JacKuo14/Documents/Metis/course_work/project03_baseball/sportstradar_data2/*game*.pkl')

In [3]:
# Create Dataframe
df_game = pd.DataFrame()
for file in gamefile_list:
    df_game = df_game.append(read_pickle(file))
    
df_play = pd.DataFrame()
for file in player_file_list:
    df_play = df_play.append(read_pickle(file))

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
df_game.shape

(12596, 809)

In [5]:
df_game.head()

,game.attendance,game.away.abbr,game.away.errors,game.away.hits,game.away.id,game.away.lineup,game.away.loss,game.away.market,game.away.name,game.away.players,game.away.probable_pitcher.era,game.away.probable_pitcher.first_name,game.away.probable_pitcher.id,game.away.probable_pitcher.jersey_number,game.away.probable_pitcher.last_name,game.away.probable_pitcher.loss,game.away.probable_pitcher.preferred_name,game.away.probable_pitcher.win,game.away.roster,game.away.runs,game.away.scoring,game.away.starting_pitcher.era,game.away.starting_pitcher.first_name,game.away.starting_pitcher.id,game.away.starting_pitcher.jersey_number,game.away.starting_pitcher.last_name,game.away.starting_pitcher.loss,game.away.starting_pitcher.preferred_name,game.away.starting_pitcher.win,game.away.statistics.fielding.overall.a,game.away.statistics.fielding.overall.assists.outfield,game.away.statistics.fielding.overall.assists.total,game.away.statistics.fielding.overall.c_wp,game.away.statistics.fielding.overall.dp,game.away.statistics.fielding.overall.error,game.away.statistics.fielding.overall.errors.fielding,game.away.statistics.fielding.overall.errors.interference,game.away.statistics.fielding.overall.errors.throwing,game.away.statistics.fielding.overall.errors.total,game.away.statistics.fielding.overall.fpct,game.away.statistics.fielding.overall.pb,game.away.statistics.fielding.overall.po,game.away.statistics.fielding.overall.steal.caught,game.away.statistics.fielding.overall.steal.pct,game.away.statistics.fielding.overall.steal.pickoff,game.away.statistics.fielding.overall.steal.stolen,game.away.statistics.fielding.overall.tc,game.away.statistics.fielding.overall.tp,game.away.statistics.hitting.overall.ab,game.away.statistics.hitting.overall.ab_risp,game.away.statistics.hitting.overall.abhr,game.away.statistics.hitting.overall.abk,game.away.statistics.hitting.overall.ap,game.away.statistics.hitting.overall.avg,game.away.statistics.hitting.overall.babip,game.away.statistics.hitting.overall.bbk,game.away.statistics.hitting.overall.bbpa,game.away.statistics.hitting.overall.bip,game.away.statistics.hitting.overall.flyball,game.away.statistics.hitting.overall.gofo,game.away.statistics.hitting.overall.groundball,game.away.statistics.hitting.overall.hit_risp,game.away.statistics.hitting.overall.iso,game.away.statistics.hitting.overall.linedrive,game.away.statistics.hitting.overall.lob,game.away.statistics.hitting.overall.lob_risp_2out,game.away.statistics.hitting.overall.obp,game.away.statistics.hitting.overall.onbase.bb,game.away.statistics.hitting.overall.onbase.cycle,game.away.statistics.hitting.overall.onbase.d,game.away.statistics.hitting.overall.onbase.fc,game.away.statistics.hitting.overall.onbase.h,game.away.statistics.hitting.overall.onbase.hbp,game.away.statistics.hitting.overall.onbase.hr,game.away.statistics.hitting.overall.onbase.ibb,game.away.statistics.hitting.overall.onbase.roe,game.away.statistics.hitting.overall.onbase.s,game.away.statistics.hitting.overall.onbase.t,game.away.statistics.hitting.overall.onbase.tb,game.away.statistics.hitting.overall.ops,game.away.statistics.hitting.overall.outcome.ball,game.away.statistics.hitting.overall.outcome.dirtball,game.away.statistics.hitting.overall.outcome.foul,game.away.statistics.hitting.overall.outcome.iball,game.away.statistics.hitting.overall.outcome.klook,game.away.statistics.hitting.overall.outcome.kswing,game.away.statistics.hitting.overall.outcome.ktotal,game.away.statistics.hitting.overall.outs.fidp,game.away.statistics.hitting.overall.outs.fo,game.away.statistics.hitting.overall.outs.gidp,game.away.statistics.hitting.overall.outs.go,game.away.statistics.hitting.overall.outs.klook,game.away.statistics.hitting.overall.outs.kswing,game.away.statistics.hitting.overall.outs.ktotal,game.away.statistics.hitting.overall.outs.lidp,game.away.statistics.hitting.overall.outs.lo,game.away.statistics.hitting.overall.outs.po,game.away.statistics.hitting.overall.outs.sacfly,game.away.statistics.hitt

In [14]:
df_play.head()

,game_id,home_away,preferred_name,first_name,last_name,plyr_id,plyr_position,plyr_primary_position,era,whip,k9,pitch_count,obp,ops,slg,rbi,avg,av_risp,hit_risp,runs,error,fpct,assists
0,083801cc-cf2c-43f1-b78a-27a5d270403d,home,Alex,Alexander,Guerrero,03e46dc5-9e2e-4493-8614-e8b1c11a0c21,OF,LF,NaN,NaN,NaN,NaN,NaN,5.000,4.000,2.0,1.000,NaN,NaN,1.0,NaN,NaN,NaN
1,083801cc-cf2c-43f1-b78a-27a5d270403d,home,Juan,Juan,Uribe,081677e0-4134-4f69-a4a4-05acf060ead0,IF,3B,NaN,NaN,NaN,NaN,NaN,1.000,0.000,0.0,.000,NaN,NaN,0.0,NaN,NaN,NaN
2,083801cc-cf2c-43f1-b78a-27a5d270403d,home,Adrián,Adrián,González,10154eef-8834-48e0-97e7-d7436367534c,IF,1B,NaN,NaN,NaN,NaN,NaN,0.800,0.400,1.0,.400,NaN,NaN,0.0,0.0,1.0,NaN
3,083801cc-cf2c-43f1-b78a-27a5d270403d,home,A.J.,Andrew,Ellis,1b59c902-b26d-464d-895c-a08b971347c2,C,C,NaN,NaN,NaN,NaN,NaN,0.250,0.000,0.0,.000,NaN,NaN,1.0,0.0,1.0,NaN
4,083801cc-cf2c-43f1-b78a-27a5d270403d,home,Howie,Howie,Kendrick,1da8f60d-0741-4b61-811c-1acca32b4393,OF,LF,NaN,NaN,NaN,NaN,NaN,0.933,0.333,0.0,.333,NaN,NaN,0.0,0.0,1.0,NaN


# Create list of Team and Team ID

In [6]:
team_df = df_game[['game.away.id', 'game.away.name']].drop_duplicates()[:30].reset_index(drop = True).rename(columns = {'game.away.id': 'team.id', 'game.away.name': 'team.name'})
#df_game[['game.home.id', 'game.home.name']].drop_duplicates()


In [7]:
league_dict = {
    'Braves': 'National',
    'Phillies': 'National',
    'White Sox': 'American',
    'Rangers': 'American',
    'Marlins': 'National',
    'Red Sox': 'American',
    'Rockies': 'National',
    'Nationals': 'National',
    'Royals': 'American',
    'Tigers': 'American',
    'Mariners': 'American',
    'Padres': 'National',
    'Giants': 'National',
    'Diamondbacks': 'National',
    'Astros': 'American',
    'Indians': 'American',
    'Pirates': 'National',
    'Yankees': 'American',
    'Blue Jays': 'American',
    'Dodgers': 'National',
    'Rays': 'American',
    'Orioles': 'American',
    'Brewers': 'National',
    'Mets': 'National',
    'Cubs': 'National',
    'Twins': 'American',
    'Athletics': 'American',
    'Reds': 'National',
    'Cardinals': 'National',
    'Angels': 'American'
}

df_league = pd.DataFrame.from_dict(league_dict, orient = 'index').reset_index().rename(columns = {'index': 'team_name', 0: 'team_league'})

In [8]:
df_league.head()

,team_name,team_league
0,Braves,National
1,Phillies,National
2,White Sox,American
3,Rangers,American
4,Marlins,National


In [9]:
team_list = team_df['team.name'].to_list()


In [10]:
# Remove games played by teams not in the MLB
mask = ((df_game['game.away.name'].isin(team_list)) 
        & (df_game['game.home.name'].isin(team_list)))
#df_game[mask].shape
df_game = df_game[mask]

# Was 12,596 games before mask
df_game.shape

In [51]:
# Add Columns
df_game = 
df_league.head()

,team_name,team_league
0,Braves,National
1,Phillies,National
2,White Sox,American
3,Rangers,American
4,Marlins,National


In [28]:
#df_game[df_game['game.away.id'] == 'dd59d49e-caee-4443-9220-f05d0d9bd1e1']
#df_game.isnull()
#df_game['game.away.name'].value_counts()
#df_game['game.home.name'].value_counts()

# Data Aggregation

In [15]:
# convert game date to date format
df_game['game.date'] = pd.to_datetime(df_game['game.scheduled']).dt.date

# Take only the columns we want
df_game_sub = df_game[['game.id',
         'game.scheduled', 
         'game.date',
        'game.home.runs', 'game.away.runs','game.away.lineup', 
         'game.home.lineup', 
         'game.home_team', 'game.home.name',
         'game.away_team', 'game.away.name', 
         'game.attendance',
                       'game.away.starting_pitcher.era', 
                       'game.home.starting_pitcher.era', 
                       'game.away.starting_pitcher.win', 
                       'game.away.starting_pitcher.loss', 
                       'game.home.starting_pitcher.loss', 
                       'game.home.starting_pitcher.win']]

df_game_sub = df_game_sub.reset_index(drop = True)

# If the home team won, make this column's value 1, if lost, then 0
df_game_sub['game.winning.team'] = np.where(df_game_sub['game.home.runs'] - df_game_sub['game.away.runs'] > 0, '1', '0')
# Percentage of games the home and away team pitchers have won
df_game_sub['game.away.pitching.winpercentage'] = df_game_sub['game.away.starting_pitcher.win'] / (df_game_sub['game.away.starting_pitcher.win'] + df_game_sub['game.away.starting_pitcher.loss'])
df_game_sub['game.home.pitching.winpercentage'] = df_game_sub['game.home.starting_pitcher.win'] / (df_game_sub['game.home.starting_pitcher.win'] + df_game_sub['game.home.starting_pitcher.loss'])

In [16]:
# 
df_game_sub = pd.merge(df_game_sub, df_league, left_on = 'game.away.name', right_on = 'team_name').rename(columns = {'team_league': 'away.team.league'}).drop('team_name', axis = 1)
df_game_sub = pd.merge(df_game_sub, df_league, left_on = 'game.home.name', right_on = 'team_name').rename(columns = {'team_league': 'home.team.league'}).drop('team_name', axis = 1)

In [17]:
df_game_sub.head()


,game.id,game.scheduled,game.date,game.home.runs,game.away.runs,game.away.lineup,game.home.lineup,game.home_team,game.home.name,game.away_team,game.away.name,game.attendance,game.away.starting_pitcher.era,game.home.starting_pitcher.era,game.away.starting_pitcher.win,game.away.starting_pitcher.loss,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.winning.team,game.away.pitching.winpercentage,game.home.pitching.winpercentage,away.team.league,home.team.league
0,083801cc-cf2c-43f1-b78a-27a5d270403d,2015-05-26T00:10:00+00:00,2015-05-26,6,3,[{'id': 'e43136cd-762f-4b3f-8595-4f07e83be3b2'...,[{'id': '43a9d631-5673-4059-9b25-d59290bc32c3'...,ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,12079497-e414-450a-8bf2-29f91de646bf,Braves,44680.0,6.136,3.614,0.0,0.0,2.0,2.0,1,NaN,0.500000,National,National
1,7ea3f5f4-fb8d-429f-9d1e-bb2c1f62b29e,2015-05-27T02:10:00+00:00,2015-05-27,8,0,[{'id': 'e43136cd-762f-4b3f-8595-4f07e83be3b2'...,[{'id': '43a9d631-5673-4059-9b25-d59290bc32c3'...,ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,12079497-e414-450a-8bf2-29f91de646bf,Braves,40667.0,3.908,4.320,4.0,1.0,3.0,2.0,1,0.800000,0.400000,National,National
2,1c8a2da6-bd91-4b68-8f91-8ff695865091,2015-05-28T02:10:00+00:00,2015-05-28,2,3,[{'id': 'e43136cd-762f-4b3f-8595-4f07e83be3b2'...,[{'id': '43a9d631-5673-4059-9b25-d59290bc32c3'...,ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,12079497-e414-450a-8bf2-29f91de646bf,Braves,37837.0,3.830,1.475,2.0,2.0,1.0,5.0,0,0.500000,0.833333,National,National
3,aa8d97d6-f474-4d59-8339-9c2ff70860eb,2016-06-04T02:10:00+00:00,2016-06-04,4,2,[{'id': '91406228-8d10-4a6d-877a-c5ae235ccc00'...,[{'id': '16c0d07f-94f0-4fec-8cfa-4703ec344865'...,ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,12079497-e414-450a-8bf2-29f91de646bf,Braves,46366.0,2.766,3.000,1.0,5.0,3.0,4.0,1,0.166667,0.571429,National,National
4,12f9f140-2752-415c-b87a-a0545f172772,2016-06-05T02:10:00+00:00,2016-06-05,4,0,[{'id': '91406228-8d10-4a6d-877a-c5ae235ccc00'...,[{'id': '16c0d07f-94f0-4fec-8cfa-4703ec344865'...,ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,12079497-e414-450a-8bf2-29f91de646bf,Braves,47126.0,5.707,1.558,1.0,6.0,1.0,7.0,1,0.142857,0.875000,National,National


In [18]:
df_game_sub['game.winning.team'].value_counts()

1    6728
0    5856
Name: game.winning.team, dtype: int64

# Add Aggregate Stats

In [19]:
path = '/Users/JacKuo14/Documents/Metis/course_work/project03_baseball/sportstradar_data2/player_stats/'
away1 = glob.glob(path + '*away1*.pkl')
away7 = glob.glob(path + '*away7*.pkl')
away30 = glob.glob(path + '*away30*.pkl')

home1 = glob.glob(path + '*home1*.pkl')
home7 = glob.glob(path + '*home7*.pkl')
home30 = glob.glob(path + '*home30*.pkl')

In [20]:
# AWAY - Avg last game
df_away1 = pd.DataFrame()
for file in away1:
    df_away1 = df_away1.append(read_pickle(file))
df_away1_group = df_away1.groupby('game_id').mean().reset_index() # groupby game_id to find mean of values for each game
df_away1_group.columns = [str(col) + '_away' for col in df_away1_group.columns]
df_away1_group = df_away1_group.rename(columns = {'game_id_away': 'game.id'})

# AWAY - Avg last 7 games
df_away7 = pd.DataFrame()
for file in away7:
    df_away7 = df_away7.append(read_pickle(file))
df_away7_group = df_away7.groupby('game_id').mean().reset_index()
df_away7_group.columns = [str(col) + '_away' for col in df_away7_group.columns]
df_away7_group = df_away7_group.rename(columns = {'game_id_away': 'game.id'})

                                       
# AWAY - Avg last 30 game
df_away30 = pd.DataFrame()
for file in away30:
    df_away30 = df_away30.append(read_pickle(file))
df_away30_group = df_away30.groupby('game_id').mean().reset_index()
df_away30_group.columns = [str(col) + '_away' for col in df_away30_group.columns]
df_away30_group = df_away30_group.rename(columns = {'game_id_away': 'game.id'})
                                       

# -----------------------------------

# HOME - Avg last game
df_home1 = pd.DataFrame()
for file in away1:
    df_home1 = df_home1.append(read_pickle(file))
df_home1_group = df_home1.groupby('game_id').mean().reset_index() # groupby game_id to find mean of values for each game
df_home1_group.columns = [str(col) + '_home' for col in df_home1_group.columns]
df_home1_group = df_home1_group.rename(columns = {'game_id_home': 'game.id'})

# HOME - Avg last 7 games
df_home7 = pd.DataFrame()
for file in away7:
    df_home7 = df_home7.append(read_pickle(file))
df_home7_group = df_home7.groupby('game_id').mean().reset_index()
df_home7_group.columns = [str(col) + '_home' for col in df_home7_group.columns]
df_home7_group = df_home7_group.rename(columns = {'game_id_home': 'game.id'})
                                       
# HOME - Avg last 30 game
df_home30 = pd.DataFrame()
for file in away30:
    df_home30 = df_home30.append(read_pickle(file))
df_home30_group = df_home30.groupby('game_id').mean().reset_index()
df_home30_group.columns = [str(col) + '_home' for col in df_home30_group.columns]
df_home30_group = df_home30_group.rename(columns = {'game_id_home': 'game.id'})

In [21]:
group_df_list = [df_away1_group, df_away7_group, df_away30_group, df_home1_group, df_home7_group, df_home30_group]

df_game_sub_merged = df_game_sub

for df in group_df_list:
    df_game_sub_merged = pd.merge(df_game_sub_merged, df, how = 'left', on = 'game.id')

In [22]:
df_game_sub_merged.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12584 entries, 0 to 12583
Columns: 113 entries, game.id to whip_30_home
dtypes: float64(99), int64(2), object(12)
memory usage: 10.9+ MB


# Investivating Zeros

In [ ]:
df_game[df_game['game.date'] == ''

In [ ]:
df_home30.head()

In [ ]:
df_game[df_game['game.date'] == '2015-05-03']

In [ ]:
df_test2 = pd.merge(df_home30, df_game[['game.id', 'game.date']], how = 'left', left_on ='game_id', right_on = 'game.id').sort_values('game.date')
df_test2[df_test2['game.date'] == '2015-05-03']

In [ ]:
df_home30_group.tail()
test_null = pd.merge(df_home30_group, df_game[['game.id', 'game.date']], how = 'left', on ='game.id')
test_null.sort_values(by = 'game.date')
#test_null[test_null['game.date'] == '2016-10-02']
test_null[test_null['game.date'] == '2015-05-01']
test_null[test_null['game.id'] == '882e476e-4be9-4b6d-b3ee-624c38b3ac1c']

In [ ]:
date(2016, 12, 31)

In [ ]:
missing_dates_list = set(df_game_sub_merged[df_game_sub_merged
                    .isnull()
                    .any(axis = 1)][df_game_sub_merged['game.date'] < date(2016, 12, 31)].sort_values('game.date')['game.date'].to_list())

In [ ]:
df_game_sub_merged[df_game_sub_merged.isnull()
                    .any(axis = 1)][df_game_sub_merged['game.date'] < date(2016, 12, 31)].sort_values('game.date')

In [ ]:
df_game[df_game['game.id'] == '882e476e-4be9-4b6d-b3ee-624c38b3ac1c']

In [ ]:
#df_game_sub_merged[df_game_sub_merged['game.date'] == date(2015, 5, 5)]
#df_game[df_game['game.date'] == date(2015, 5, 5)]['game.away.players'][1]
df_game[df_game['game.id'] == '882e476e-4be9-4b6d-b3ee-624c38b3ac1c']['game.away.players'].reset_index(drop = True)[0]


In [ ]:
missing_dates_list

In [ ]:
# Remove columns that are completely blank
df_game_sub_merged = df_game_sub_merged.dropna(axis=1, how='all')

In [ ]:
df_game_sub_merged.columns

# Baseline Model

In [ ]:
# all_agg_cols produced test score of 54% accuracy


# Baseline with all features that have more than 1000 rows filled in

In [ ]:
# all_agg_cols produced test score of 54% accuracy for Random Forest


In [23]:
cols_to_drop = ['obp_1_away', 'obp_7_away', 'obp_30_away', 
'av_risp_1_away', 'av_risp_7_away', 'av_risp_30_away',
'hit_risp_1_away', 'hit_risp_7_away', 'hit_risp_30_away',
'obp_1_home', 'obp_7_home', 'obp_30_home',
'av_risp_1_home', 'av_risp_7_home', 'av_risp_30_home',
'hit_risp_1_home', 'hit_risp_7_home', 'hit_risp_30_home',
'game.scheduled', 'game.home.runs', 'game.away.runs', 
'game.away.lineup', 'game.home.lineup', 'game.home_team', 'game.away_team', 'game.id', 'game.attendance',
'assists_1_away', 'assists_7_away', 'avg_7_away', 'assists_30_away', 'avg_30_away',
'assists_1_home', 'assists_7_home', 'avg_7_home', 'assists_30_home', 'avg_30_home', 'game.date', 'game.home.name', 'game.away.name']


df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
#df_game_sub_merged[df_game_sub_merged.isnull().any(axis = 1)]


# Create target
target = df_baseline['game.winning.team']
df_baseline.drop(labels=['game.winning.team'], axis=1,inplace = True)
df_baseline.insert(0, 'game.winning.team', target)


#df_baseline.head()
#df_baseline.info() 

In [24]:
all_agg_cols = df_baseline.columns.to_list()


In [25]:
df_baseline = df_baseline.dropna().reset_index(drop = True)
df_baseline.head()

,game.winning.team,game.away.starting_pitcher.era,game.home.starting_pitcher.era,game.away.starting_pitcher.win,game.away.starting_pitcher.loss,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.away.pitching.winpercentage,game.home.pitching.winpercentage,away.team.league,home.team.league,avg_1_away,era_1_away,error_1_away,fpct_1_away,k9_1_away,ops_1_away,pitch_count_1_away,rbi_1_away,runs_1_away,slg_1_away,whip_1_away,era_7_away,error_7_away,fpct_7_away,k9_7_away,ops_7_away,pitch_count_7_away,rbi_7_away,runs_7_away,slg_7_away,whip_7_away,era_30_away,error_30_away,fpct_30_away,k9_30_away,ops_30_away,pitch_count_30_away,rbi_30_away,runs_30_away,slg_30_away,whip_30_away,avg_1_home,era_1_home,error_1_home,fpct_1_home,k9_1_home,ops_1_home,pitch_count_1_home,rbi_1_home,runs_1_home,slg_1_home,whip_1_home,era_7_home,error_7_home,fpct_7_home,k9_7_home,ops_7_home,pitch_count_7_home,rbi_7_home,runs_7_home,slg_7_home,whip_7_home,era_30_home,error_30_home,fpct_30_home,k9_30_home,ops_30_home,pitch_count_30_home,rbi_30_home,runs_30_home,slg_30_home,whip_30_home
0,1,2.766,3.000,1.0,5.0,3.0,4.0,0.166667,0.571429,National,National,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790
1,1,5.707,1.558,1.0,6.0,1.0,7.0,0.142857,0.875000,National,National,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420
2,1,3.161,4.382,2.0,5.0,3.0,4.0,0.285714,0.571429,National,National,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374
3,0,3.844,3.375,7.0,5.0,3.0,6.0,0.583333,0.666667,National,National,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488
4,0,4.330,1.558,3.0,7.0,0.0,11.0,0.300000,1.000000,National,National,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843613,6.732000,0.731955,21.266667,0.400000,0.384848,0.411352,1.529563,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538

In [26]:
# Create dummy columns for away and home team leagues
df_baseline = pd.get_dummies(df_baseline, columns = ['away.team.league', 'home.team.league'])


In [27]:
df_baseline['game.winning.team'].value_counts()


1    4242
0    3851
Name: game.winning.team, dtype: int64

In [28]:
df_baseline.head()

,game.winning.team,game.away.starting_pitcher.era,game.home.starting_pitcher.era,game.away.starting_pitcher.win,game.away.starting_pitcher.loss,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.away.pitching.winpercentage,game.home.pitching.winpercentage,avg_1_away,era_1_away,error_1_away,fpct_1_away,k9_1_away,ops_1_away,pitch_count_1_away,rbi_1_away,runs_1_away,slg_1_away,whip_1_away,era_7_away,error_7_away,fpct_7_away,k9_7_away,ops_7_away,pitch_count_7_away,rbi_7_away,runs_7_away,slg_7_away,whip_7_away,era_30_away,error_30_away,fpct_30_away,k9_30_away,ops_30_away,pitch_count_30_away,rbi_30_away,runs_30_away,slg_30_away,whip_30_away,avg_1_home,era_1_home,error_1_home,fpct_1_home,k9_1_home,ops_1_home,pitch_count_1_home,rbi_1_home,runs_1_home,slg_1_home,whip_1_home,era_7_home,error_7_home,fpct_7_home,k9_7_home,ops_7_home,pitch_count_7_home,rbi_7_home,runs_7_home,slg_7_home,whip_7_home,era_30_home,error_30_home,fpct_30_home,k9_30_home,ops_30_home,pitch_count_30_home,rbi_30_home,runs_30_home,slg_30_home,whip_30_home,away.team.league_American,away.team.league_National,home.team.league_American,home.team.league_National
0,1,2.766,3.000,1.0,5.0,3.0,4.0,0.166667,0.571429,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790,0,1,0,1
1,1,5.707,1.558,1.0,6.0,1.0,7.0,0.142857,0.875000,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420,0,1,0,1
2,1,3.161,4.382,2.0,5.0,3.0,4.0,0.285714,0.571429,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374,0,1,0,1
3,0,3.844,3.375,7.0,5.0,3.0,6.0,0.583333,0.666667,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488,0,1,0,1
4,0,4.330,1.558,3.0,7.0,0.0,11.0,0.300000,1.000000,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843613,6.732000,0.731955,21.266667,0.400000,0.384848,0.411352,1.529563,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153

# Random Forest Baseline Model

In [29]:
def home_predict(x_array):
    return(['1' for x in range(len(x_array))])

def away_predict(x_array):
    return(['0' for x in range(len(x_array))])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline.iloc[:, 0], 
                                                    test_size = 0.2, random_state=42)

In [31]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [32]:
## RANDOM FOREST ##
rf = RandomForestClassifier(n_estimators = 600, max_depth = 8)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)

print("train accuracy: ", accuracy_score(y_train, rf.predict(X_train)))
print("test accuracy: ", accuracy_score(y_test, y_predict))
print("Home Model Accuracy: ", accuracy_score(y_test, home_predict(y_test)))


train accuracy:  0.8668520234785295
test accuracy:  0.5719579987646696
Home Model Accuracy:  0.525633106856084


In [34]:
print("Away Model Accuracy: ", accuracy_score(y_test, away_predict(y_test)))


Away Model Accuracy:  0.474366893143916


In [35]:
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(rf, X_train, y_train, cv = 10))


0.5486597904166167

In [36]:
feature_importances = pd.DataFrame(rf.feature_importances_,index = X_train.columns,columns=['importance']).sort_values('importance', ascending=False)

In [37]:
feature_importances[:20]


,importance
game.away.starting_pitcher.era,0.022934
game.home.starting_pitcher.era,0.022095
game.home.starting_pitcher.win,0.020387
game.away.pitching.winpercentage,0.019087
game.home.pitching.winpercentage,0.018636
k9_30_away,0.018140
k9_30_home,0.017747
era_30_home,0.017469
game.away.starting_pitcher.win,0.017393
error_30_home,0.016639


In [59]:
### RANDOM FOREST with just pitcher data ### 
# This has more rows of data.
game_cols_to_drop = ['game.id', 'game.scheduled', 'game.date', 'game.home.runs',
       'game.away.runs', 'game.away.lineup', 'game.home.lineup',
       'game.home_team', 'game.home.name', 'game.away_team', 'game.away.name',
       'game.attendance']

df_game_sub1 = df_game_sub.drop(game_cols_to_drop, axis = 1)

target = df_game_sub1['game.winning.team']
df_game_sub1.drop(labels=['game.winning.team'], axis=1,inplace = True)
df_game_sub1.insert(0, 'game.winning.team', target)
df_game_sub1 = pd.get_dummies(df_game_sub1, columns = ['away.team.league', 'home.team.league'])
df_game_sub1 = df_game_sub1.dropna()

In [39]:
df_game_sub1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772 entries, 1 to 12583
Data columns (total 13 columns):
game.winning.team                   10772 non-null object
game.away.starting_pitcher.era      10772 non-null float64
game.home.starting_pitcher.era      10772 non-null float64
game.away.starting_pitcher.win      10772 non-null float64
game.away.starting_pitcher.loss     10772 non-null float64
game.home.starting_pitcher.loss     10772 non-null float64
game.home.starting_pitcher.win      10772 non-null float64
game.away.pitching.winpercentage    10772 non-null float64
game.home.pitching.winpercentage    10772 non-null float64
away.team.league_American           10772 non-null uint8
away.team.league_National           10772 non-null uint8
home.team.league_American           10772 non-null uint8
home.team.league_National           10772 non-null uint8
dtypes: float64(8), object(1), uint8(4)
memory usage: 883.6+ KB


In [40]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df_game_sub1.iloc[:, 1:], df_game_sub1.iloc[:, 0], 
                                                    test_size = 0.2, random_state=42)

rf = RandomForestClassifier(n_estimators = 600, max_depth = 8)
rf.fit(X_train1, y_train1)
y_predict1 = rf.predict(X_test1)

print("train accuracy: ", accuracy_score(y_train1, rf.predict(X_train1)))
print("test accuracy: ", accuracy_score(y_test1, y_predict1))

print("Home Model Accuracy: ", accuracy_score(y_test1, home_predict(y_test1)))


train accuracy:  0.7032609957061622
test accuracy:  0.5605568445475638
Home Model Accuracy:  0.5433874709976798


In [41]:
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)
print("The score for kNN is")
print("Training: {:6.2f}%".format(100*knn.score(X_train, y_train)))
print("Test set: {:6.2f}%".format(100*knn.score(X_test, y_test)))
print("Home Model Accuracy: ", accuracy_score(y_test, home_predict(y_test)))


The score for kNN is
Training:  64.46%
Test set:  50.96%
Home Model Accuracy:  0.525633106856084


In [42]:
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()

#X_tr = std_scale(X_train)

X_tr = std_scale.fit_transform(X_train)
X_te = std_scale.transform(X_test)

logit = LogisticRegression(C = 10)
logit.fit(X_tr, y_train)
print("The score for logistic regression is")
print("Training: {:6.2f}%".format(100*logit.score(X_tr, y_train)))
print("Test set: {:6.2f}%".format(100*logit.score(X_te, y_test)))
print("Home Model Accuracy: ", accuracy_score(y_test, home_predict(y_test)))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The score for logistic regression is
Training:  55.84%
Test set:  55.53%
Home Model Accuracy:  0.525633106856084


In [ ]:
y_test.astype('int')


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

fpr, tpr, thresholds = roc_curve(np.array(y_test.astype('int')), rf.predict_proba(X_test)[:,1])


In [ ]:
plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])


plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for fraud problem');
print("ROC AUC score = ", roc_auc_score(np.array(y_test.astype('int')), rf.predict_proba(X_test)[:,1]))

In [52]:
from sklearn.metrics import accuracy_score
import xgboost as xgb


In [43]:
### XGBOOST ###
###
df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)


In [44]:
df_baseline.head()


,game.away.starting_pitcher.era,game.home.starting_pitcher.era,game.away.starting_pitcher.win,game.away.starting_pitcher.loss,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.winning.team,game.away.pitching.winpercentage,game.home.pitching.winpercentage,away.team.league,home.team.league,avg_1_away,era_1_away,error_1_away,fpct_1_away,k9_1_away,ops_1_away,pitch_count_1_away,rbi_1_away,runs_1_away,slg_1_away,whip_1_away,era_7_away,error_7_away,fpct_7_away,k9_7_away,ops_7_away,pitch_count_7_away,rbi_7_away,runs_7_away,slg_7_away,whip_7_away,era_30_away,error_30_away,fpct_30_away,k9_30_away,ops_30_away,pitch_count_30_away,rbi_30_away,runs_30_away,slg_30_away,whip_30_away,avg_1_home,era_1_home,error_1_home,fpct_1_home,k9_1_home,ops_1_home,pitch_count_1_home,rbi_1_home,runs_1_home,slg_1_home,whip_1_home,era_7_home,error_7_home,fpct_7_home,k9_7_home,ops_7_home,pitch_count_7_home,rbi_7_home,runs_7_home,slg_7_home,whip_7_home,era_30_home,error_30_home,fpct_30_home,k9_30_home,ops_30_home,pitch_count_30_home,rbi_30_home,runs_30_home,slg_30_home,whip_30_home
0,6.136,3.614,0.0,0.0,2.0,2.0,1,NaN,0.500000,National,National,0.210643,5.512500,0.076923,0.839154,18.3375,0.522571,31.000000,0.071429,0.142857,0.228500,2.975000,2.035714,0.010989,0.697168,6.750000,0.455286,12.952381,0.252747,0.252747,0.213176,1.314286,NaN,0.038889,0.930890,NaN,0.670067,NaN,0.383333,0.466667,0.350911,NaN,0.210643,5.512500,0.076923,0.839154,18.3375,0.522571,31.000000,0.071429,0.142857,0.228500,2.975000,2.035714,0.010989,0.697168,6.750000,0.455286,12.952381,0.252747,0.252747,0.213176,1.314286,NaN,0.038889,0.930890,NaN,0.670067,NaN,0.383333,0.466667,0.350911,NaN
1,3.908,4.320,4.0,1.0,3.0,2.0,1,0.800000,0.400000,National,National,0.233300,9.234000,0.000000,0.800000,11.2815,0.533300,85.000000,0.300000,0.300000,0.258300,1.305200,6.120357,0.028571,0.868229,7.720071,0.488295,91.928571,0.319048,0.285714,0.224262,1.476593,NaN,0.046667,0.935313,NaN,0.702640,NaN,0.420000,0.520000,0.365093,NaN,0.233300,9.234000,0.000000,0.800000,11.2815,0.533300,85.000000,0.300000,0.300000,0.258300,1.305200,6.120357,0.028571,0.868229,7.720071,0.488295,91.928571,0.319048,0.285714,0.224262,1.476593,NaN,0.046667,0.935313,NaN,0.702640,NaN,0.420000,0.520000,0.365093,NaN
2,3.830,1.475,2.0,2.0,1.0,5.0,0,0.500000,0.833333,National,National,0.102538,0.428667,0.000000,0.818182,15.4290,0.352538,43.666667,0.076923,0.000000,0.121769,0.761900,3.507952,0.026374,0.744327,10.260857,0.487147,43.142857,0.210623,0.219780,0.224711,1.529024,NaN,0.038889,0.902885,NaN,0.695622,NaN,0.388889,0.483333,0.363222,NaN,0.102538,0.428667,0.000000,0.818182,15.4290,0.352538,43.666667,0.076923,0.000000,0.121769,0.761900,3.507952,0.026374,0.744327,10.260857,0.487147,43.142857,0.210623,0.219780,0.224711,1.529024,NaN,0.038889,0.902885,NaN,0.695622,NaN,0.388889,0.483333,0.363222,NaN
3,2.766,3.000,1.0,5.0,3.0,4.0,1,0.166667,0.571429,National,National,0.120333,3.552500,0.100000,0.783300,3.5505,0.490667,63.000000,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.3461,0.627252,100.5,0.388889,0.377778,0.349548,1.46579,0.120333,3.552500,0.100000,0.783300,3.5505,0.490667,63.000000,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.3461,0.627252,100.5,0.388889,0.377778,0.349548,1.46579
4,5.707,1.558,1.0,6.0,1.0,7.0,1,0.142857,0.875000,National,National,0.326143,6.394667,0.062500,0.562500,0.9480,0.809500,49.000000,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.9553,0.665419,43.7,0.325000,0.387500,0.381785,1.23342,0.326143,6.394667,0.062500,0.562500,0.9480,0.809500,49.000000,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.0263

In [60]:
df_baseline = df_game_sub_merged.drop(columns = cols_to_drop + ["away.team.league", "home.team.league"])
target = df_baseline['game.winning.team']
df_baseline.drop(labels=['game.winning.team'], axis=1,inplace = True)
df_baseline.insert(0, 'game.winning.team', target)
df_baseline = df_baseline.dropna()


In [61]:
df_baseline.head()

,game.winning.team,game.away.starting_pitcher.era,game.home.starting_pitcher.era,game.away.starting_pitcher.win,game.away.starting_pitcher.loss,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.away.pitching.winpercentage,game.home.pitching.winpercentage,avg_1_away,era_1_away,error_1_away,fpct_1_away,k9_1_away,ops_1_away,pitch_count_1_away,rbi_1_away,runs_1_away,slg_1_away,whip_1_away,era_7_away,error_7_away,fpct_7_away,k9_7_away,ops_7_away,pitch_count_7_away,rbi_7_away,runs_7_away,slg_7_away,whip_7_away,era_30_away,error_30_away,fpct_30_away,k9_30_away,ops_30_away,pitch_count_30_away,rbi_30_away,runs_30_away,slg_30_away,whip_30_away,avg_1_home,era_1_home,error_1_home,fpct_1_home,k9_1_home,ops_1_home,pitch_count_1_home,rbi_1_home,runs_1_home,slg_1_home,whip_1_home,era_7_home,error_7_home,fpct_7_home,k9_7_home,ops_7_home,pitch_count_7_home,rbi_7_home,runs_7_home,slg_7_home,whip_7_home,era_30_home,error_30_home,fpct_30_home,k9_30_home,ops_30_home,pitch_count_30_home,rbi_30_home,runs_30_home,slg_30_home,whip_30_home
3,1,2.766,3.000,1.0,5.0,3.0,4.0,0.166667,0.571429,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790
4,1,5.707,1.558,1.0,6.0,1.0,7.0,0.142857,0.875000,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420
5,1,3.161,4.382,2.0,5.0,3.0,4.0,0.285714,0.571429,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374
6,0,3.844,3.375,7.0,5.0,3.0,6.0,0.583333,0.666667,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488
7,0,4.330,1.558,3.0,7.0,0.0,11.0,0.300000,1.000000,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843613,6.732000,0.731955,21.266667,0.400000,0.384848,0.411352,1.529563,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843

In [62]:
df_baseline['game.winning.team']

3        1
4        1
5        1
6        0
7        0
        ..
12577    0
12578    1
12581    1
12582    0
12583    0
Name: game.winning.team, Length: 8093, dtype: object

In [63]:
df_baseline.iloc[:, 1:].head()

,game.away.starting_pitcher.era,game.home.starting_pitcher.era,game.away.starting_pitcher.win,game.away.starting_pitcher.loss,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.away.pitching.winpercentage,game.home.pitching.winpercentage,avg_1_away,era_1_away,error_1_away,fpct_1_away,k9_1_away,ops_1_away,pitch_count_1_away,rbi_1_away,runs_1_away,slg_1_away,whip_1_away,era_7_away,error_7_away,fpct_7_away,k9_7_away,ops_7_away,pitch_count_7_away,rbi_7_away,runs_7_away,slg_7_away,whip_7_away,era_30_away,error_30_away,fpct_30_away,k9_30_away,ops_30_away,pitch_count_30_away,rbi_30_away,runs_30_away,slg_30_away,whip_30_away,avg_1_home,era_1_home,error_1_home,fpct_1_home,k9_1_home,ops_1_home,pitch_count_1_home,rbi_1_home,runs_1_home,slg_1_home,whip_1_home,era_7_home,error_7_home,fpct_7_home,k9_7_home,ops_7_home,pitch_count_7_home,rbi_7_home,runs_7_home,slg_7_home,whip_7_home,era_30_home,error_30_home,fpct_30_home,k9_30_home,ops_30_home,pitch_count_30_home,rbi_30_home,runs_30_home,slg_30_home,whip_30_home
3,2.766,3.000,1.0,5.0,3.0,4.0,0.166667,0.571429,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790
4,5.707,1.558,1.0,6.0,1.0,7.0,0.142857,0.875000,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420
5,3.161,4.382,2.0,5.0,3.0,4.0,0.285714,0.571429,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374
6,3.844,3.375,7.0,5.0,3.0,6.0,0.583333,0.666667,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488
7,4.330,1.558,3.0,7.0,0.0,11.0,0.300000,1.000000,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843613,6.732000,0.731955,21.266667,0.400000,0.384848,0.411352,1.529563,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843613,6.732000,0.731955,21.266

In [64]:
df_baseline.head()

,game.winning.team,game.away.starting_pitcher.era,game.home.starting_pitcher.era,game.away.starting_pitcher.win,game.away.starting_pitcher.loss,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.away.pitching.winpercentage,game.home.pitching.winpercentage,avg_1_away,era_1_away,error_1_away,fpct_1_away,k9_1_away,ops_1_away,pitch_count_1_away,rbi_1_away,runs_1_away,slg_1_away,whip_1_away,era_7_away,error_7_away,fpct_7_away,k9_7_away,ops_7_away,pitch_count_7_away,rbi_7_away,runs_7_away,slg_7_away,whip_7_away,era_30_away,error_30_away,fpct_30_away,k9_30_away,ops_30_away,pitch_count_30_away,rbi_30_away,runs_30_away,slg_30_away,whip_30_away,avg_1_home,era_1_home,error_1_home,fpct_1_home,k9_1_home,ops_1_home,pitch_count_1_home,rbi_1_home,runs_1_home,slg_1_home,whip_1_home,era_7_home,error_7_home,fpct_7_home,k9_7_home,ops_7_home,pitch_count_7_home,rbi_7_home,runs_7_home,slg_7_home,whip_7_home,era_30_home,error_30_home,fpct_30_home,k9_30_home,ops_30_home,pitch_count_30_home,rbi_30_home,runs_30_home,slg_30_home,whip_30_home
3,1,2.766,3.000,1.0,5.0,3.0,4.0,0.166667,0.571429,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790,0.120333,3.552500,0.100000,0.783300,3.55050,0.490667,63.0,0.111111,0.111111,0.231444,1.168400,2.451714,0.085714,0.813814,6.574500,0.839921,59.142857,0.587302,0.587302,0.450508,1.092321,4.133000,0.064211,0.864867,7.346100,0.627252,100.500000,0.388889,0.377778,0.349548,1.465790
4,1,5.707,1.558,1.0,6.0,1.0,7.0,0.142857,0.875000,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420,0.326143,6.394667,0.062500,0.562500,0.94800,0.809500,49.0,0.428571,0.500000,0.411857,1.328933,6.176857,0.026786,0.691068,7.926000,0.568876,44.761905,0.295238,0.328571,0.304590,1.402024,4.686067,0.026345,0.734352,7.955300,0.665419,43.700000,0.325000,0.387500,0.381785,1.233420
5,1,3.161,4.382,2.0,5.0,3.0,4.0,0.285714,0.571429,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374,0.089214,11.314200,0.076923,0.589769,16.32780,0.452286,28.4,0.214286,0.214286,0.255929,2.171420,3.185400,0.010204,0.642861,10.402457,0.496735,33.914286,0.244898,0.295918,0.308663,1.110077,4.083700,0.023660,0.629899,9.667320,0.483990,25.726667,0.259524,0.259694,0.275804,1.364374
6,0,3.844,3.375,7.0,5.0,3.0,6.0,0.583333,0.666667,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488,0.202357,4.500000,0.000000,0.714286,7.12575,0.535714,34.5,0.285714,0.285714,0.279786,1.166675,1.638714,0.040816,0.705687,11.270250,0.586204,34.464286,0.316327,0.326531,0.346082,1.050136,3.498714,0.046707,0.654162,7.781164,0.500024,28.353810,0.241667,0.254762,0.281420,1.385488
7,0,4.330,1.558,3.0,7.0,0.0,11.0,0.300000,1.000000,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843613,6.732000,0.731955,21.266667,0.400000,0.384848,0.411352,1.529563,0.262846,0.794000,0.076923,0.918538,8.46900,0.615385,61.5,0.307692,0.153846,0.320538,1.470600,5.119500,0.056777,0.769861,5.282357,0.549615,53.642857,0.351648,0.285714,0.292846,1.584293,4.487500,0.050925,0.843

In [66]:
#df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline['game.winning.team'], 
                                                    test_size = 0.2, random_state=40, stratify = df_baseline['game.winning.team'])


In [67]:
gbm = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=6,
                        objective='binary:logistic', #new objective
                        learning_rate=.05, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8
                       )

eval_set=[(X_train,y_train),(X_test,y_test)]
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=50,
                    verbose=False
                   )

accuracy_score(y_test, gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit))
#np.mean(cross_val_score(gbm, X_train, y_train, cv = 5))


0.5917232859789994

In [68]:
gbm2 = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=6,
                        objective='binary:logistic', #new objective
                        learning_rate=.05, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8, reg_lambda = 1.1
                       )

eval_set=[(X_train,y_train),(X_test,y_test)]
fit_model = gbm2.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=50,
                    verbose=False
                   )

print("Test Accuracy Score: ", accuracy_score(y_test, gbm2.predict(X_test, ntree_limit=gbm2.best_ntree_limit)))
#print("Cross Validation Score: ", np.mean(cross_val_score(gbm2, X_train, y_train, cv = 5)))


Test Accuracy Score:  0.58307597282273


In [69]:
# Highest Test Accuracy was with train/test split, random state = 39
gbm2 = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=5,
                        objective='binary:logistic', #new objective
                        learning_rate=.05, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8, reg_lambda = 1.1
                       )

eval_set=[(X_train,y_train),(X_test,y_test)]
fit_model = gbm2.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=20,
                    verbose=False
                   )

print("Test Accuracy Score: ", accuracy_score(y_test, gbm2.predict(X_test, ntree_limit=gbm2.best_ntree_limit)))
#print("Cross Validation Score: ", np.mean(cross_val_score(gbm2, X_train, y_train, cv = 5)))

Test Accuracy Score:  0.5892526250772081


In [70]:
print("Train Accuracy Score: ", accuracy_score(y_train, gbm2.predict(X_train, ntree_limit=gbm2.best_ntree_limit)))

Train Accuracy Score:  0.764287920914427


In [71]:
print("Baseline Accuracy Score: ", accuracy_score(y_test, home_predict(y_test)))

Baseline Accuracy Score:  0.5243977764051884


In [228]:
# k-fold cross validation evaluation of xgboost model

from sklearn.model_selection import KFold

# CV model
#model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, random_state=39)
results = cross_val_score(gbm2, X_train, y_train, cv=kfold)
np.mean(results)
#print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

KeyboardInterrupt: 

In [314]:
### LIGHT GBM ###

import lightgbm as lgb

#df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline['game.winning.team'].astype('int'), 
                                                    test_size = 0.2, random_state=40, stratify = df_baseline['game.winning.team'])


d_train = lgb.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.001
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['sub_feature'] = 0.5
params['num_leaves'] = 35
params['min_data'] = 20
params['max_depth'] = 10
params['metric'] = 'binary_logloss'
params['num_iterations'] = 600
#params['lambda_l2'] = 0.2

clf = lgb.train(params, d_train, 100)

#Prediction
y_pred_1 = clf.predict(X_test)

y_pred_te = []
for i in range(len(y_pred_1)):
    if y_pred_1[i]>=.5:       # setting threshold to .
        y_pred_te.append(1)
    else:  
        y_pred_te.append(0)

#Prediction
y_pred_2 = clf.predict(X_train)

y_pred_tr = []
for i in range(len(y_pred_2)):
    if y_pred_2[i]>=.5:       # setting threshold to .
        y_pred_tr.append(1)
    else:  
        y_pred_tr.append(0)

print("Test Accuracy Score: ", accuracy_score(y_test, y_pred_te)) # 0.56 with no reg
print("Train Accuracy Score: ", accuracy_score(y_train, y_pred_tr))
baseline_predict = [int(x) for x in home_predict(y_test) ]
print("Baseline Accuracy Score: ", accuracy_score(y_test, baseline_predict))

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Test Accuracy Score:  0.5631067961165048
Train Accuracy Score:  0.7517006802721088
Baseline Accuracy Score:  0.5223300970873787


In [315]:
### LIGHT GBM, with Regularization ###

d_train = lgb.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.001
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['sub_feature'] = 0.5
params['num_leaves'] = 35
params['min_data'] = 20
params['max_depth'] = 10
params['metric'] = 'binary_logloss'
params['num_iterations'] = 600
params['lambda_l2'] = 0.2

clf = lgb.train(params, d_train, 100)

#Prediction
y_pred_1 = clf.predict(X_test)

y_pred_te = []
for i in range(len(y_pred_1)):
    if y_pred_1[i]>=.5:       # setting threshold to .
        y_pred_te.append(1)
    else:  
        y_pred_te.append(0)

#Prediction
y_pred_2 = clf.predict(X_train)

y_pred_tr = []
for i in range(len(y_pred_2)):
    if y_pred_2[i]>=.5:       # setting threshold to .
        y_pred_tr.append(1)
    else:  
        y_pred_tr.append(0)


print("Test Accuracy Score: ", accuracy_score(y_test, y_pred_te)) # 0.56 with no reg
print("Train Accuracy Score: ", accuracy_score(y_train, y_pred_tr))
baseline_predict = [int(x) for x in home_predict(y_test) ]
print("Baseline Accuracy Score: ", accuracy_score(y_test, baseline_predict))

Test Accuracy Score:  0.5611650485436893
Train Accuracy Score:  0.7514577259475219
Baseline Accuracy Score:  0.5223300970873787


In [287]:
int(home_predict(y_test))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [271]:
y_test

416     1
4821    1
4096    1
2312    1
447     0
       ..
279     1
3879    1
2541    1
2201    1
1726    1
Name: game.winning.team, Length: 1030, dtype: int64

In [278]:
y_pred_2

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,


# Boosting with Categorical Variables

In [383]:
cols_to_drop = ['obp_1_away', 'obp_7_away', 'obp_30_away', 
'av_risp_1_away', 'av_risp_7_away', 'av_risp_30_away',
'hit_risp_1_away', 'hit_risp_7_away', 'hit_risp_30_away',
'obp_1_home', 'obp_7_home', 'obp_30_home',
'av_risp_1_home', 'av_risp_7_home', 'av_risp_30_home',
'hit_risp_1_home', 'hit_risp_7_home', 'hit_risp_30_home',
'game.scheduled', 'game.home.runs', 'game.away.runs', 
'game.away.lineup', 'game.home.lineup', 'game.home_team', 'game.away_team', 'game.id', 'game.attendance',
'assists_1_away', 'assists_7_away', 'avg_7_away', 'assists_30_away', 'avg_30_away',
'assists_1_home', 'assists_7_home', 'avg_7_home', 'assists_30_home', 'avg_30_home']


df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
#df_game_sub_merged[df_game_sub_merged.isnull().any(axis = 1)]


# Create target
target = df_baseline['game.winning.team']
df_baseline.drop(labels=['game.winning.team'], axis=1,inplace = True)
df_baseline.insert(0, 'game.winning.team', target)


#df_baseline.head()
#df_baseline.info() 

In [384]:
df_baseline['game.week.day'] = pd.to_datetime(df_baseline['game.date']).dt.day_name()
df_baseline = df_baseline.drop('game.date', axis = 1)

In [385]:
df_baseline = df_baseline.dropna().reset_index(drop = True)
df_baseline = pd.get_dummies(df_baseline, columns = ['game.home.name', 'game.away.name', 'away.team.league', 'home.team.league', 'game.week.day'])

In [386]:
#df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline['game.winning.team'], 
                                                    test_size = 0.2, random_state=40, stratify = df_baseline['game.winning.team'])


In [387]:
### XGBOOST with CATEGORICAL VARIABLES ###
gbm2 = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=6,
                        objective='binary:logistic', #new objective
                        learning_rate=.05, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8, reg_lambda = 1.1
                       )

eval_set=[(X_train,y_train),(X_test,y_test)]
fit_model = gbm2.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=50,
                    verbose=False
                   )

print("Test Accuracy Score: ", accuracy_score(y_test, gbm2.predict(X_test, ntree_limit=gbm2.best_ntree_limit)))
print("Train Accuracy Score: ", accuracy_score(y_train, gbm2.predict(X_train, ntree_limit=gbm2.best_ntree_limit)))
print("Baseline Accuracy Score: ", accuracy_score(y_test, home_predict(y_test)))
#print("Cross Validation Score: ", np.mean(cross_val_score(gbm2, X_train, y_train, cv = 5)))

Test Accuracy Score:  0.5805825242718446
Train Accuracy Score:  0.8734207968901846
Baseline Accuracy Score:  0.5223300970873787


In [391]:
feature_importances = pd.DataFrame(gbm2.feature_importances_,index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
#len(feature_importances)
feature_importances[-50:]



,importance
game.week.day_Wednesday,0.007540
runs_1_home,0.007513
away.team.league_American,0.007420
home.team.league_American,0.007419
game.away.starting_pitcher.loss,0.007367
game.away.name_Rangers,0.007289
k9_1_away,0.007234
game.home.name_Yankees,0.007150
rbi_30_home,0.007138
whip_7_home,0.007109


In [350]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4116 entries, 1792 to 581
Columns: 141 entries, game.away.starting_pitcher.era to game.week.day_Wednesday
dtypes: float64(70), uint8(71)
memory usage: 2.5 MB


In [361]:
### LIGHT GBM ###
#df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline['game.winning.team'].astype('int'), 
                                                    test_size = 0.2, random_state=40, stratify = df_baseline['game.winning.team'])

d_train = lgb.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.001
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['sub_feature'] = 0.5
params['num_leaves'] = 35
params['min_data'] = 20
params['max_depth'] = 10
params['metric'] = 'binary_logloss'
params['num_iterations'] = 600
#params['lambda_l2'] = 0.2

clf = lgb.train(params, d_train, 100)

#Prediction
y_pred_1 = clf.predict(X_test)

y_pred_te = []
for i in range(len(y_pred_1)):
    if y_pred_1[i]>=.5:       # setting threshold to .
        y_pred_te.append(1)
    else:  
        y_pred_te.append(0)

#Prediction
y_pred_2 = clf.predict(X_train)

y_pred_tr = []
for i in range(len(y_pred_2)):
    if y_pred_2[i]>=.5:       # setting threshold to .
        y_pred_tr.append(1)
    else:  
        y_pred_tr.append(0)

print("Test Accuracy Score: ", accuracy_score(y_test, y_pred_te)) # 0.56 with no reg
print("Train Accuracy Score: ", accuracy_score(y_train, y_pred_tr))
baseline_predict = [int(x) for x in home_predict(y_test) ]
print("Baseline Accuracy Score: ", accuracy_score(y_test, baseline_predict))

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Test Accuracy Score:  0.5572815533980583
Train Accuracy Score:  0.7517006802721088
Baseline Accuracy Score:  0.5223300970873787


In [362]:
### CAT BOOST ###

df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
#df_game_sub_merged[df_game_sub_merged.isnull().any(axis = 1)]


# Create target
target = df_baseline['game.winning.team']
df_baseline.drop(labels=['game.winning.team'], axis=1,inplace = True)
df_baseline.insert(0, 'game.winning.team', target)

df_baseline['game.week.day'] = pd.to_datetime(df_baseline['game.date']).dt.day_name()
df_baseline = df_baseline.drop('game.date', axis = 1)
df_baseline = df_baseline.dropna().reset_index(drop = True)

In [377]:
#df_baseline.head()
from catboost import CatBoostClassifier

X = df_baseline.iloc[:, 1:]
y = df_baseline['game.winning.team']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=40, stratify = df_baseline['game.winning.team'])
categorical_features_indices = np.where(X.dtypes != np.float)[0]

cat = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

cat.fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_test,y_test))
cat.score(X_test, y_test)

print("Test Accuracy Score: ", accuracy_score(y_test.astype('int'), cat.predict(X_test))) # 0.56 with no reg
print("Train Accuracy Score: ", accuracy_score(y_train.astype('int'), cat.predict(X_train)))
baseline_predict = [int(x) for x in home_predict(y_test) ]
print("Baseline Accuracy Score: ", accuracy_score(y_test.astype('int'), baseline_predict))

#cat.predict(X_train)

Learning rate set to 0.090526
0:	learn: 0.5432459	test: 0.5213592	best: 0.5213592 (0)	total: 89.1ms	remaining: 1m 29s
1:	learn: 0.5626822	test: 0.5262136	best: 0.5262136 (1)	total: 164ms	remaining: 1m 21s
2:	learn: 0.5801749	test: 0.5310680	best: 0.5310680 (2)	total: 237ms	remaining: 1m 18s
3:	learn: 0.5952381	test: 0.5359223	best: 0.5359223 (3)	total: 329ms	remaining: 1m 21s
4:	learn: 0.6003401	test: 0.5262136	best: 0.5359223 (3)	total: 400ms	remaining: 1m 19s
5:	learn: 0.6044704	test: 0.5184466	best: 0.5359223 (3)	total: 459ms	remaining: 1m 16s
6:	learn: 0.6064140	test: 0.5145631	best: 0.5359223 (3)	total: 552ms	remaining: 1m 18s
7:	learn: 0.6171040	test: 0.5145631	best: 0.5359223 (3)	total: 674ms	remaining: 1m 23s
8:	learn: 0.6260933	test: 0.5203883	best: 0.5359223 (3)	total: 763ms	remaining: 1m 23s
9:	learn: 0.6343537	test: 0.5281553	best: 0.5359223 (3)	total: 832ms	remaining: 1m 22s
10:	learn: 0.6435860	test: 0.5116505	best: 0.5359223 (3)	total: 927ms	remaining: 1m 23s
11:	learn: 

94:	learn: 0.8241011	test: 0.5456311	best: 0.5524272 (32)	total: 6.53s	remaining: 1m 2s
95:	learn: 0.8243440	test: 0.5485437	best: 0.5524272 (32)	total: 6.58s	remaining: 1m 2s
96:	learn: 0.8267736	test: 0.5446602	best: 0.5524272 (32)	total: 6.64s	remaining: 1m 1s
97:	learn: 0.8309038	test: 0.5446602	best: 0.5524272 (32)	total: 6.68s	remaining: 1m 1s
98:	learn: 0.8304179	test: 0.5504854	best: 0.5524272 (32)	total: 6.72s	remaining: 1m 1s
99:	learn: 0.8330904	test: 0.5504854	best: 0.5524272 (32)	total: 6.79s	remaining: 1m 1s
100:	learn: 0.8330904	test: 0.5524272	best: 0.5524272 (32)	total: 6.85s	remaining: 1m
101:	learn: 0.8364917	test: 0.5504854	best: 0.5524272 (32)	total: 6.9s	remaining: 1m
102:	learn: 0.8362488	test: 0.5514563	best: 0.5524272 (32)	total: 6.96s	remaining: 1m
103:	learn: 0.8386783	test: 0.5582524	best: 0.5582524 (103)	total: 6.99s	remaining: 1m
104:	learn: 0.8420797	test: 0.5563107	best: 0.5582524 (103)	total: 7.04s	remaining: 1m
105:	learn: 0.8442663	test: 0.5524272	bes

186:	learn: 0.9205539	test: 0.5563107	best: 0.5601942 (106)	total: 10.6s	remaining: 46.1s
187:	learn: 0.9207969	test: 0.5563107	best: 0.5601942 (106)	total: 10.7s	remaining: 46s
188:	learn: 0.9234694	test: 0.5514563	best: 0.5601942 (106)	total: 10.7s	remaining: 45.9s
189:	learn: 0.9237123	test: 0.5533981	best: 0.5601942 (106)	total: 10.8s	remaining: 45.9s
190:	learn: 0.9232264	test: 0.5553398	best: 0.5601942 (106)	total: 10.8s	remaining: 45.8s
191:	learn: 0.9234694	test: 0.5563107	best: 0.5601942 (106)	total: 10.9s	remaining: 45.7s
192:	learn: 0.9232264	test: 0.5563107	best: 0.5601942 (106)	total: 10.9s	remaining: 45.6s
193:	learn: 0.9241983	test: 0.5563107	best: 0.5601942 (106)	total: 10.9s	remaining: 45.5s
194:	learn: 0.9246842	test: 0.5592233	best: 0.5601942 (106)	total: 11s	remaining: 45.3s
195:	learn: 0.9256560	test: 0.5611650	best: 0.5611650 (195)	total: 11s	remaining: 45.2s
196:	learn: 0.9266278	test: 0.5621359	best: 0.5621359 (196)	total: 11.1s	remaining: 45.1s
197:	learn: 0.92

278:	learn: 0.9676871	test: 0.5543689	best: 0.5631068 (198)	total: 16.1s	remaining: 41.7s
279:	learn: 0.9674441	test: 0.5553398	best: 0.5631068 (198)	total: 16.2s	remaining: 41.6s
280:	learn: 0.9693878	test: 0.5504854	best: 0.5631068 (198)	total: 16.2s	remaining: 41.5s
281:	learn: 0.9703596	test: 0.5495146	best: 0.5631068 (198)	total: 16.3s	remaining: 41.4s
282:	learn: 0.9703596	test: 0.5485437	best: 0.5631068 (198)	total: 16.3s	remaining: 41.3s
283:	learn: 0.9703596	test: 0.5524272	best: 0.5631068 (198)	total: 16.3s	remaining: 41.2s
284:	learn: 0.9701166	test: 0.5475728	best: 0.5631068 (198)	total: 16.4s	remaining: 41.1s
285:	learn: 0.9706025	test: 0.5475728	best: 0.5631068 (198)	total: 16.4s	remaining: 41s
286:	learn: 0.9710884	test: 0.5514563	best: 0.5631068 (198)	total: 16.5s	remaining: 40.9s
287:	learn: 0.9708455	test: 0.5524272	best: 0.5631068 (198)	total: 16.5s	remaining: 40.8s
288:	learn: 0.9706025	test: 0.5524272	best: 0.5631068 (198)	total: 16.5s	remaining: 40.7s
289:	learn: 

372:	learn: 0.9880952	test: 0.5553398	best: 0.5631068 (198)	total: 20.1s	remaining: 33.8s
373:	learn: 0.9885811	test: 0.5543689	best: 0.5631068 (198)	total: 20.1s	remaining: 33.7s
374:	learn: 0.9883382	test: 0.5533981	best: 0.5631068 (198)	total: 20.2s	remaining: 33.6s
375:	learn: 0.9880952	test: 0.5553398	best: 0.5631068 (198)	total: 20.2s	remaining: 33.5s
376:	learn: 0.9880952	test: 0.5533981	best: 0.5631068 (198)	total: 20.2s	remaining: 33.4s
377:	learn: 0.9888241	test: 0.5572816	best: 0.5631068 (198)	total: 20.3s	remaining: 33.4s
378:	learn: 0.9893100	test: 0.5592233	best: 0.5631068 (198)	total: 20.3s	remaining: 33.3s
379:	learn: 0.9893100	test: 0.5640777	best: 0.5640777 (379)	total: 20.4s	remaining: 33.2s
380:	learn: 0.9888241	test: 0.5621359	best: 0.5640777 (379)	total: 20.4s	remaining: 33.1s
381:	learn: 0.9885811	test: 0.5582524	best: 0.5640777 (379)	total: 20.4s	remaining: 33.1s
382:	learn: 0.9885811	test: 0.5553398	best: 0.5640777 (379)	total: 20.5s	remaining: 33s
383:	learn: 

...

560:	learn: 0.9980564	test: 0.5601942	best: 0.5640777 (379)	total: 28.1s	remaining: 22s
561:	learn: 0.9980564	test: 0.5611650	best: 0.5640777 (379)	total: 28.2s	remaining: 21.9s
562:	learn: 0.9980564	test: 0.5592233	best: 0.5640777 (379)	total: 28.2s	remaining: 21.9s
563:	learn: 0.9980564	test: 0.5592233	best: 0.5640777 (379)	total: 28.2s	remaining: 21.8s
564:	learn: 0.9980564	test: 0.5592233	best: 0.5640777 (379)	total: 28.3s	remaining: 21.8s
565:	learn: 0.9980564	test: 0.5611650	best: 0.5640777 (379)	total: 28.3s	remaining: 21.7s
566:	learn: 0.9980564	test: 0.5592233	best: 0.5640777 (379)	total: 28.3s	remaining: 21.6s
567:	learn: 0.9980564	test: 0.5592233	best: 0.5640777 (379)	total: 28.4s	remaining: 21.6s
568:	learn: 0.9980564	test: 0.5592233	best: 0.5640777 (379)	total: 28.4s	remaining: 21.5s
569:	learn: 0.9980564	test: 0.5592233	best: 0.5640777 (379)	total: 28.5s	remaining: 21.5s
570:	learn: 0.9980564	test: 0.5582524	best: 0.5640777 (379)	total: 28.5s	remaining: 21.4s
571:	learn: 

654:	learn: 0.9992711	test: 0.5592233	best: 0.5660194 (584)	total: 32.1s	remaining: 16.9s
655:	learn: 0.9992711	test: 0.5572816	best: 0.5660194 (584)	total: 32.1s	remaining: 16.8s
656:	learn: 0.9992711	test: 0.5572816	best: 0.5660194 (584)	total: 32.1s	remaining: 16.8s
657:	learn: 0.9992711	test: 0.5572816	best: 0.5660194 (584)	total: 32.2s	remaining: 16.7s
658:	learn: 0.9992711	test: 0.5601942	best: 0.5660194 (584)	total: 32.2s	remaining: 16.7s
659:	learn: 0.9992711	test: 0.5611650	best: 0.5660194 (584)	total: 32.2s	remaining: 16.6s
660:	learn: 0.9992711	test: 0.5601942	best: 0.5660194 (584)	total: 32.3s	remaining: 16.6s
661:	learn: 0.9992711	test: 0.5601942	best: 0.5660194 (584)	total: 32.3s	remaining: 16.5s
662:	learn: 0.9992711	test: 0.5601942	best: 0.5660194 (584)	total: 32.4s	remaining: 16.5s
663:	learn: 0.9992711	test: 0.5611650	best: 0.5660194 (584)	total: 32.4s	remaining: 16.4s
664:	learn: 0.9992711	test: 0.5582524	best: 0.5660194 (584)	total: 32.4s	remaining: 16.3s
665:	learn

747:	learn: 0.9997570	test: 0.5514563	best: 0.5660194 (584)	total: 36s	remaining: 12.1s
748:	learn: 0.9997570	test: 0.5495146	best: 0.5660194 (584)	total: 36.1s	remaining: 12.1s
749:	learn: 0.9997570	test: 0.5553398	best: 0.5660194 (584)	total: 36.1s	remaining: 12s
750:	learn: 0.9997570	test: 0.5543689	best: 0.5660194 (584)	total: 36.1s	remaining: 12s
751:	learn: 0.9997570	test: 0.5543689	best: 0.5660194 (584)	total: 36.2s	remaining: 11.9s
752:	learn: 0.9997570	test: 0.5504854	best: 0.5660194 (584)	total: 36.2s	remaining: 11.9s
753:	learn: 0.9997570	test: 0.5514563	best: 0.5660194 (584)	total: 36.3s	remaining: 11.8s
754:	learn: 0.9997570	test: 0.5533981	best: 0.5660194 (584)	total: 36.4s	remaining: 11.8s
755:	learn: 0.9997570	test: 0.5524272	best: 0.5660194 (584)	total: 36.4s	remaining: 11.7s
756:	learn: 0.9997570	test: 0.5514563	best: 0.5660194 (584)	total: 36.4s	remaining: 11.7s
757:	learn: 0.9997570	test: 0.5504854	best: 0.5660194 (584)	total: 36.5s	remaining: 11.6s
758:	learn: 0.99

843:	learn: 1.0000000	test: 0.5572816	best: 0.5660194 (584)	total: 40.1s	remaining: 7.41s
844:	learn: 1.0000000	test: 0.5572816	best: 0.5660194 (584)	total: 40.1s	remaining: 7.36s
845:	learn: 1.0000000	test: 0.5582524	best: 0.5660194 (584)	total: 40.2s	remaining: 7.31s
846:	learn: 1.0000000	test: 0.5582524	best: 0.5660194 (584)	total: 40.2s	remaining: 7.26s
847:	learn: 1.0000000	test: 0.5563107	best: 0.5660194 (584)	total: 40.2s	remaining: 7.21s
848:	learn: 1.0000000	test: 0.5592233	best: 0.5660194 (584)	total: 40.3s	remaining: 7.16s
849:	learn: 1.0000000	test: 0.5592233	best: 0.5660194 (584)	total: 40.3s	remaining: 7.12s
850:	learn: 1.0000000	test: 0.5553398	best: 0.5660194 (584)	total: 40.4s	remaining: 7.07s
851:	learn: 1.0000000	test: 0.5572816	best: 0.5660194 (584)	total: 40.4s	remaining: 7.02s
852:	learn: 1.0000000	test: 0.5563107	best: 0.5660194 (584)	total: 40.5s	remaining: 6.98s
853:	learn: 1.0000000	test: 0.5563107	best: 0.5660194 (584)	total: 40.5s	remaining: 6.93s
854:	learn

938:	learn: 1.0000000	test: 0.5572816	best: 0.5660194 (584)	total: 44.1s	remaining: 2.86s
939:	learn: 1.0000000	test: 0.5582524	best: 0.5660194 (584)	total: 44.1s	remaining: 2.82s
940:	learn: 1.0000000	test: 0.5592233	best: 0.5660194 (584)	total: 44.2s	remaining: 2.77s
941:	learn: 1.0000000	test: 0.5592233	best: 0.5660194 (584)	total: 44.2s	remaining: 2.72s
942:	learn: 1.0000000	test: 0.5582524	best: 0.5660194 (584)	total: 44.3s	remaining: 2.67s
943:	learn: 1.0000000	test: 0.5601942	best: 0.5660194 (584)	total: 44.3s	remaining: 2.63s
944:	learn: 1.0000000	test: 0.5640777	best: 0.5660194 (584)	total: 44.3s	remaining: 2.58s
945:	learn: 1.0000000	test: 0.5640777	best: 0.5660194 (584)	total: 44.4s	remaining: 2.53s
946:	learn: 1.0000000	test: 0.5640777	best: 0.5660194 (584)	total: 44.4s	remaining: 2.49s
947:	learn: 1.0000000	test: 0.5650485	best: 0.5660194 (584)	total: 44.5s	remaining: 2.44s
948:	learn: 1.0000000	test: 0.5640777	best: 0.5660194 (584)	total: 44.5s	remaining: 2.39s
949:	learn

In [374]:
type(cat.predict(X_test)[0])

numpy.float64

In [376]:
y_test.astype('int')

416     1
4821    1
4096    1
2312    1
447     0
       ..
279     1
3879    1
2541    1
2201    1
1726    1
Name: game.winning.team, Length: 1030, dtype: int64

In [ ]:
X = 
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [366]:
np.where(X.dtypes != np.float)[0]

array([ 0,  1, 10, 11, 74])

In [ ]:
gbm.score(X_test, y_test)


In [ ]:
gbm.score(X_train, y_train)

In [ ]:
import lightgbm as lgb

In [ ]:
#df_baseline = df_game_sub_merged.drop(columns = cols_to_drop)
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline['game.winning.team'], 
                                                    test_size = 0.2, random_state=39)

# making lgbm datasets for train and valid
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_test, y_test)

In [ ]:
params = {
    'objective' :'binary',
    'learning_rate' : 0.02,
    'num_leaves' : 76,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'binary_logloss'
}

bst = lgb.train(params, 
                 X_train, 
                 5000, 
                 valid_sets=[d_valid], 
                 verbose_eval=50, 
                 early_stopping_rounds=100)




#lgb.cv(param, X_train, num_round, nfold=5)


In [ ]:
###
from catboost import CatBoostClassifier


In [ ]:
X_tr_cat = X_train.astype('int')


In [ ]:
X_tr_cat.head()


In [ ]:
X_te_cat = X_test.astype('int')


In [ ]:
cat_features = [0, 1]

model = CatBoostClassifier(iterations=300,
                           learning_rate=0.1,
                           depth=8)
# Fit model
model.fit(X_tr_cat, y_train, cat_features)
# Get predicted classes
preds_class = model.predict(X_te_cat)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(X_te_cat)
# Get predicted RawFormulaVal
#preds_raw = model.predict(X_te_cat, prediction_type='RawFormulaVal')


In [ ]:
y_test = y_test.astype('int')


In [ ]:
accuracy_score(y_test, model.predict(X_te_cat))


In [ ]:
y_test


In [ ]:
preds_class

In [ ]:
pd.DataFrame(y_test, preds_class)

# Try with top 25 features in previous model

In [ ]:
top_25_features = feature_importances.index.to_list()
top_25_features.insert(0, "game.winning.team")


In [ ]:
df_baseline[top_25_features]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline.iloc[:, 0], 
                                                    test_size = 0.2, random_state=42)
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)

print("train accuracy: ", accuracy_score(y_train, rf.predict(X_train)))
print("test accuracy: ", accuracy_score(y_test, y_predict))

feature_importances = pd.DataFrame(rf.feature_importances_,index = X_train.columns,columns=['importance']).sort_values('importance', ascending=False)

In [ ]:
feature_importances


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [ ]:
df_baseline = df_game_sub_merged[['game.winning.team','rbi_30_home', 'runs_30_home', 'slg_30_home', 'whip_30_home', 'k9_30_home', 'rbi_30_away', 'runs_30_away', 'slg_30_away', 'whip_30_away', 'k9_30_away']]

In [ ]:
len(df_baseline)


In [ ]:
null_games = df_game_sub_merged[df_game_sub_merged['rbi_30_home'].isnull()]['game.id'].to_list()


In [ ]:
df_game.info()

In [ ]:
df_game[df_game['game.id'].isin(null_games)]

In [ ]:
df_baseline[df_baseline['rbi_30_home'].isnull()]

In [ ]:
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline.iloc[:, 0], 
                                                    test_size = 0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)

In [ ]:
drop_cols = ['rbi_30_home', 'runs_30_home', 'slg_30_home', 'whip_30_home', 'k9_30_home', 'rbi_30_away', 'runs_30_away', 'slg_30_away', 'whip_30_away', 'k9_30_away']
df_baseline = df_baseline.dropna(subset=drop_cols)

In [ ]:
df_baseline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_baseline.iloc[:, 1:], df_baseline.iloc[:, 0], 
                                                    test_size = 0.2, random_state=42)

In [ ]:
import seaborn as sns
#train_df = X_train.copy()
#train_df['home_win'] = y_train
#sns.pairplot(train_df, hue='home_win')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df_score = pd.DataFrame(y_predict, y_test).reset_index()
df_score['correct?'] = np.where(df_score['game.winning.team'] == df_score[0], "yes", "no")
df_score['correct?'].value_counts()


In [ ]:
247/len(df_score)


In [ ]:
rf_confusion 


In [ ]:
rf_confusion = confusion_matrix(y_test, rf.predict(X_test))
plt.figure(dpi=150)
sns.heatmap(rf_confusion, cmap=plt.cm.Blues, annot=True, square=True)
           #xticklabels=iris_dataset['target_names'],
           #yticklabels=iris_dataset['target_names'])
        

In [ ]:
df

In [ ]:
df_test.head()

In [ ]:
pd.DataFrame(df_game_sub_merged.isna().sum()).sort_values(by = 0, ascending = False).head(70)

In [ ]:
df_game_sub_merged.info(verbose=True, memory_usage='deep')

In [ ]:
df_away7_group[df_away7_group['game.id'] == '07e2d14a-8de8-48e8-92a4-efbe4a4b0921']

In [ ]:
cols = ['era_1_away',
 'whip_1_away',
 'k9_1_away',
 'pitch_count_1_away',
 'obp_1_away',
 'ops_1_away',
 'slg_1_away',
 'rbi_1_away',
 'avg_1_away',
 'av_risp_1_away',
 'hit_risp_1_away',
 'runs_1_away',
 'error_1_away',
 'fpct_1_away',
 'assists_1_away',
 'assists_7_away',
 'av_risp_7_away',
 'avg_7_away',
 'era_7_away',
 'error_7_away',
 'fpct_7_away',
 'hit_risp_7_away',
 'k9_7_away',
 'obp_7_away',
 'ops_7_away',
 'pitch_count_7_away',
 'rbi_7_away',
 'runs_7_away',
 'slg_7_away',
 'whip_7_away',
 'assists_30_away',
 'av_risp_30_away',
 'avg_30_away',
 'era_30_away',
 'error_30_away',
 'fpct_30_away',
 'hit_risp_30_away',
 'k9_30_away',
 'obp_30_away',
 'ops_30_away',
 'pitch_count_30_away',
 'rbi_30_away',
 'runs_30_away',
 'slg_30_away',
 'whip_30_away',
 'era_1_home',
 'whip_1_home',
 'k9_1_home',
 'pitch_count_1_home',
 'obp_1_home',
 'ops_1_home',
 'slg_1_home',
 'rbi_1_home',
 'avg_1_home',
 'av_risp_1_home',
 'hit_risp_1_home',
 'runs_1_home',
 'error_1_home',
 'fpct_1_home',
 'assists_1_home',
 'assists_7_home',
 'av_risp_7_home',
 'avg_7_home',
 'era_7_home',
 'error_7_home',
 'fpct_7_home',
 'hit_risp_7_home',
 'k9_7_home',
 'obp_7_home',
 'ops_7_home',
 'pitch_count_7_home',
 'rbi_7_home',
 'runs_7_home',
 'slg_7_home',
 'whip_7_home',
 'assists_30_home',
 'av_risp_30_home',
 'avg_30_home',
 'era_30_home',
 'error_30_home',
 'fpct_30_home',
 'hit_risp_30_home',
 'k9_30_home',
 'obp_30_home',
 'ops_30_home',
 'pitch_count_30_home',
 'rbi_30_home',
 'runs_30_home',
 'slg_30_home',
 'whip_30_home']

In [ ]:
df_game_sub_merged.columns.to_list()

In [ ]:
df_test = df_game_sub_merged[cols].dropna(how='all')
df_test = df_test.dropna(axis=1, how='all')
df_test.info()

In [ ]:
df_game_sub_merged.tail(20)

In [ ]:
df_away30_group.shape

In [ ]:
col_list = df_game_sub_merged.columns.to_list()

In [ ]:
col_list

In [ ]:
df_game_sub.head()

In [ ]:
df_away1_group = df_away1.groupby('game_id').mean().reset_index()

In [ ]:
gamefile_list = glob.glob('/Users/JacKuo14/Documents/Metis/course_work/project03_baseball/sportstrader_data/*game*.pkl')


In [ ]:
read_pickle()


In [ ]:
stats_list = ['era', 
              'whip',
              'k9', 
              'pitch_count', 
              'obp', 
              'ops', 'slg', 'rbi', 'avg', 'av_risp', 'hit_risp', 'runs', 'error', 'fpct', 'assists']